In [1]:
pip install PIL pdf2image boto3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from PIL import Image
from surya.ocr import run_ocr
from surya.model.detection.model import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from pdf2image import convert_from_path

def process_pdf(pdf_path, langs=["en"], dpi=300):
    # Convert PDF pages to images with higher DPI
    images = convert_from_path(pdf_path, dpi=dpi)

    # Load OCR models and processors
    det_processor, det_model = load_detection_processor(), load_detection_model()
    rec_model, rec_processor = load_recognition_model(), load_recognition_processor()

    all_predictions = []

    # Run OCR on each page image
    for image in images:
        predictions = run_ocr([image], [langs], det_model, det_processor, rec_model, rec_processor)
        all_predictions.extend(predictions)

    return all_predictions


# Example usage:
pdf_path = "E Module 02 Mock Drill Kaise Kare.pdf"
predictions = process_pdf(pdf_path)

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec on device cuda with dtype torch.float16


Recognizing Text: 100%|██████████| 1/1 [00:06<00:00,  6.04s/it]


In [9]:
def extract_plain_text(predictions):
    paragraphs = []
    current_paragraph = []

    # Threshold for determining a new paragraph based on the vertical gap between lines
    paragraph_threshold = 30

    # Track the y position of the last line
    last_y2 = None

    # Iterate over each OCRResult object in predictions
    for prediction in predictions:
        # Iterate over each TextLine object in the OCRResult
        for line in prediction.text_lines:
            # Get the y2 position (bottom of the bounding box)
            y2 = line.polygon[2][1]  # The y2 coordinate in the polygon

            # If this is the first line or if there's a significant gap, start a new paragraph
            if last_y2 is not None and y2 - last_y2 > paragraph_threshold:
                # If current_paragraph has content, join it into a paragraph
                if current_paragraph:
                    paragraphs.append(" ".join(current_paragraph))
                current_paragraph = []

            # Append the current line's text to the current paragraph
            current_paragraph.append(line.text)
            last_y2 = y2

    # Add the last paragraph to the list
    if current_paragraph:
        paragraphs.append(" ".join(current_paragraph))

    # Combine all paragraphs with double newlines for paragraph separation
    combined_text = "\n\n".join(paragraphs)

    return combined_text

# Example usage:
plain_text = extract_plain_text(predictions)
print(plain_text)


 प . सवित। : '' डुं બોલું તો મારું માથું દુ:ખી જાય છે’’ 

 संજુની મિત્ર છે સવિતા. બંને જણ એક જ વર્ગમાં સાથે ભણે છે. 

તેના પિતા ગંગાપ્રસાદ દવા બનાવવાના કારખાનમાં કામ કરે છે. તે

કહે છે, “ભૂકંપ આવ્યો એટલે હું ગભરાઈ ગઈ. મને તો ખબર નહોતી

કે ભૂકંપ એટલે શું? મારો આ પહેલો અનુભવ હતો. હું બચવા દોડી.

મારા માથામાં છંટ વાગી. ખૂબ લોહી આવ્યું અને હું બેભાન થઈ ગઈ.”

સારવાર પછી થોડુંક સારું લાગ્યું એટલે સવિતાએ તેનાં મમ્મીને કહ્યું કે મારે ભણવા

  

 તો લાગતો જ હતો. ૨૬મી જાન્યુઆરી, ૨૦૦૨ના રોજ તો તે આ ડરને કારણે શાળામાં

ગઈ જ  નહીં. જોકે તેનો ભાઈ શાળાએ ગયો હતો અને તેના ભાઈએ શાળામાં ધ્વજવંદનના

કાર્યક્રમ વિશે પછી સવિતાને કહ્યું કે “જો, તને કહ્યું તું ને, કશું થયું નહીં.”

સવિતા કહે છે, “મને ભૂકંપ વિશે કોઈએ કશું સમજાવ્યું નથી. હા, મારી મમ્મીએ કહ્યું

કે ભૂકંપ આવે તો ભાગી જવાનું, નહીંત૨ મરી જવાય.” ભૂકંપના લાંબા સમય પછી પણ

સવિતાને ઘણી તકલીફ છે. તે કહે છે, “હું વધુ બોલું તો મારું માથું દુઃખી જાય. એથી હું વધુ

બોલતી નથી. મારો ભાઈ મને માર-માર કરે એટલે હું બોલું. હું મારા બંને હાથોથી કંઈ

કામ નથી કરી શકતી. 

For Translation (AWS)

In [2]:
import boto3

def translate_text(text, source_lang='hi', target_lang='en'):
    if not text.strip():  # Check if the text is empty or just whitespace
        return ""  # Return an empty string if there's no text to translate
    
    translate = boto3.client(
        'translate',
        aws_access_key_id='',
        aws_secret_access_key='',
        region_name='us-east-1'
    )
    
    response = translate.translate_text(
        Text=text,
        SourceLanguageCode=source_lang,
        TargetLanguageCode=target_lang
    )
    
    return response['TranslatedText']

def translate_predictions(predictions):
    translated_predictions = []

    for ocr_result in predictions:  # predictions is a list of OCRResult objects
        for text_line in ocr_result.text_lines:  # Iterate over text_lines in each OCRResult
            translated_text = translate_text(text_line.text)
            translated_predictions.append({
                'original_text': text_line.text,
                'translated_text': translated_text,
                'confidence': text_line.confidence,
                'bbox': text_line.bbox
            })
    
    return translated_predictions

# Assuming `predictions` is the list of OCRResult objects
translated_predictions = translate_predictions(predictions)

# Example to print the translated predictions
for item in translated_predictions:
    print(f"Original Text: {item['original_text']}")
    print(f"Translated Text: {item['translated_text']}")
    print(f"Confidence: {item['confidence']}")
    print(f"BBox: {item['bbox']}\n")

Original Text: मॉक-ड़िल केसे करे?
Translated Text: How to mock?
Confidence: 0.8486328125
BBox: [1039.0, 102.0, 2399.0, 280.0]

Original Text: Photo: AIDMI.
Translated Text: Photo: AIDMI.
Confidence: 0.95458984375
BBox: [3212.0, 387.0, 3505.0, 432.0]

Original Text: स्कूल शिक्षा विभाग, छत्तीसगढ़ शासन इस ई–मोड्युल को बनाने हेतु यूनिसेफ, छत्तीसगढ़ की सराहना करता है।
Translated Text: School Education Department, Government of Chhattisgarh appreciates UNICEF, Chhattisgarh for creating this e-module.
Confidence: 0.966796875
BBox: [1306.0, 2254.0, 3376.0, 2320.0]

Original Text: Department of School Education 
Translated Text: Department of School Education 
Confidence: 0.99609375
BBox: [312.0, 2274.0, 950.0, 2322.0]

Original Text: 12.
Translated Text: 12.
Confidence: 0.37646484375
BBox: [1710.0, 2356.0, 1789.0, 2396.0]

Original Text: A CONTROL AND END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END 

for generating PDF

In [4]:
from pdf2image import convert_from_path
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
import textwrap

def create_translated_pdf(output_pdf_path, translated_predictions):
    c = canvas.Canvas(output_pdf_path, pagesize=A4)
    
    # A4 dimensions in points: 595 x 842 (width x height)
    page_width, page_height = A4

    # Determine the maximum possible x and y values from the bbox to calculate the scaling factor
    max_bbox_width = 2200  # Replace with the actual maximum x2 value in your dataset
    max_bbox_height = 900

    x_offset = -20
    y_offset = -400

    # Scale factor to map the bounding box coordinates to the PDF page size
    x_scale = page_width / max_bbox_width
    y_scale = page_height / max_bbox_height

    # Fixed line height (adjust this value as needed)
    line_height = 8  # Example line height in points

    # Starting y-position (from top of the page)
    current_y_position = page_height - 50  # Start 50 points down from the top

    # Minimum y-position before a new page is needed
    bottom_margin = 50  # 50 points margin from the bottom

    # Maximum width for the text before wrapping
    max_text_width = page_width - 40  # Leaving some margin on both sides

    for prediction in translated_predictions:
        # Extracting bounding box coordinates
        x1, y1, x2, y2 = prediction['bbox']

        # Scale the x and y coordinates
        scaled_x1 = x1 * x_scale + x_offset
        scaled_y1 = y1 * y_scale + y_offset

        # Calculate text position based on fixed y position
        text_x = scaled_x1
        text_y = current_y_position

        # Set font and size
        c.setFont("Helvetica", 8)

        # Wrap the text within max_text_width
        wrapped_lines = textwrap.wrap(prediction['translated_text'], width=int(max_text_width ))

        for line in wrapped_lines:
            if text_y < bottom_margin + line_height:
                # If y position is below bottom margin, start a new page
                c.showPage()
                current_y_position = page_height - 50
                text_y = current_y_position
                text_x = scaled_x1  # Reset text_x if necessary

            c.drawString(text_x, text_y, line)
            text_y -= line_height  # Move down to the next line

        # Move the current_y_position down for the next block of text
        current_y_position = text_y - line_height

    # Save the PDF
    c.save()

output_pdf_path = "translated_output.pdf"
create_translated_pdf(output_pdf_path, translated_predictions)    